# K-Prototypes Tech Talk

## Outline
* Unsupervised machine learning and what does it mean to cluster data
    * How does regular K-means work?
    * Other clustering algos
* What are its advantages and limitations
* Why is using standard distance metric problematic?
* K-prototypes
* What libraries are available?
* Application to Colombia vulnerability index

## References

* Original paper
    * https://link.springer.com/article/10.1023/A:1009769707641
* Python implementation
    * https://pypi.org/project/kmodes/
    * https://github.com/nicodv/kmodes
* Other articles and notebooks using k-prototypes
    * https://www.kaggle.com/rohanadagouda/unsupervised-learning-using-k-prototype-and-dbscan
    * https://towardsdatascience.com/the-k-prototype-as-clustering-algorithm-for-mixed-data-type-categorical-and-numerical-fe7c50538ebb

* Data for this demo
    * https://www.kaggle.com/giovamata/airlinedelaycauses
    
    
    
    
    
https://www.kaggle.com/ashydv/bank-customer-clustering-k-modes-clustering




### How does K-means work

Algorithm
* Initialize with a random set of cluster vectors (these are m-dimensional vectors where m is the number of features in the dataset)
* In a loop:
    * For each record in the dataset, calculate the Euclidean distance between the record and to each cluster vector
    * Assign each record to the cluster vector closet to it (i.e. lowest Euclidean distance)
    * For each cluster, calculate the mean of all the records assigned to that cluster (average of each component across all vectors belonging to the same cluster).
    * Replace the previous cluster vectors with the the new cluster vectors calculated in the previous step
    
Here's a visualization of the algorithm
    
    
![kmeans_algo](./imgs/kmeans_algo.png)
[Source](https://en.wikipedia.org/wiki/K-means_clustering)

### Why K-means shouldn't be used with categorical variables

Categorical variables are data that takes one of a limited and usually fixed number of possible values. If they can be ordered, then it is called `ordinal`. If it cannot be ordered, then it is called `nominal`. Here are some examples:

    * ordinal: risk rating, rank, day of the week
    * nominal: political party, region of residence, occupation

There are several ways these variables are used in models. For unsupervised ML, some approaches include one-hot encoding, label encoding, feature aggregation, or excluding the feature from the model.

For one-hot encoding, the categorical is converted to several binary features each of which representing a given category in the original feature. This works when there are few categories, but become a computational burden when categories exceeds hundres or thousands of unique values (e.g. zip codes).

Label encoding only works for ordinal data, but even in such case it would require that the encoded feature follows the same ordering as the original values in the raw feature.

In the case of a categorical feature with high cardinality, one solution is to aggregate them. For example, zip codes can be aggregated to a region containing many zip codes. Following the aggregation, one can then apply one-hot encoding.

When the approaches above fails or is infeasible to apply, the data scientist may categorical feature all together from the model.





In [1]:
# Importing required packages
import numpy as np
import pandas as pd 
import random
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./data/DelayedFlights.csv')
df = df.drop('Unnamed: 0', axis=1)

In [3]:
df.shape

(1936758, 29)

In [4]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(14), int64(10), object(5)
memory usage: 428.5+ MB


In [8]:
num_cols = df._get_numeric_data().columns
cols = df.columns
cat_cols = list(set(cols) - set(num_cols))
print(cat_cols)
print(num_cols)

['Dest', 'Origin', 'UniqueCarrier', 'TailNum', 'CancellationCode']
Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'FlightNum', 'ActualElapsedTime',
       'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Distance',
       'TaxiIn', 'TaxiOut', 'Cancelled', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')


In [11]:
df_k = df.loc[:,['Cancelled','ActualElapsedTime','TaxiOut', 'DepDelay']]

In [13]:
from sklearn import datasets

iris = datasets.load_iris()

data = np.c_[iris['data'], iris['target']]

kp = KPrototypes(n_clusters=3, init='Huang', n_init=1, verbose=True)
kp.fit_predict(data, categorical=[4])

print(kp.cluster_centroids_)
print(kp.labels_)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 19, ncost: 87.49232821349614
Run: 1, iteration: 2/100, moves: 3, ncost: 87.22715722315765
Run: 1, iteration: 3/100, moves: 0, ncost: 87.22715722315765
[[5.91568627 2.76470588 4.26470588 1.33333333 1.        ]
 [5.006      3.428      1.462      0.246      0.        ]
 [6.62244898 2.98367347 5.57346939 2.03265306 2.        ]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 0 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [15]:
data

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [5. , 3.4, 1.5, 0.2, 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ],
       [5.4, 3.7, 1.5, 0.2, 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [4.3, 3. , 1.1, 0.1, 0. ],
       [5.8, 4. , 1.2, 0.2, 0. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. ],
       [5.1, 3.5, 1.4, 0.3, 0. ],
       [5.7, 3.8, 1.7, 0.3, 0. ],
       [5.1, 3.8, 1.5, 0.3, 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. ],
       [5.1, 3.7, 1.5, 0.4, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [5.1, 3.3, 1.7, 0.5, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [5. , 3. , 1.6, 0.2, 0. ],
       [5. , 3.4, 1.6, 0.4, 0. ],
       [5.2, 3.5, 1.5, 0.2, 0. ],
       [5.2, 3.4, 1.4, 0.2, 0. ],
       [4.7, 3